In [28]:
import pandas as pd

## Pandas Dataframes

In previous years we ran this course using R, however this year we will continue to work with Python. As you have learned, the most common library for manipulating structured data in Python is called `pandas`. This notebook covers a quick refresher and tour, using some of the data we will work with later in this course.

There are a lot of pandas resources available, eg [cheatsheets](https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf).

The data that we have just loaded comes from the file containing the Arthurian manuscripts metadata. Each row represents a single manuscript and the columns describe different properties of each of these books (or at least, what remains of them).

First, let's load the data from a CSV file on disk.

In [ ]:
df = pd.read_csv("../../datasets/arthur/manuscripts.csv", index_col=0)
df

We can use the `shape` property to find the dimensions of the `DataFrame` object (rows x columns). Remember that Python *properties* are accessed as bare words (`df.shape` not `df.shape()`), whereas *methods* must have parentheses, even when no arguments are given.

In [ ]:
df.shape

There are many other useful properties. We can look at the `columns`, and later we will make a lot of use of the `index`

In [ ]:
df.columns

### Basic Methods

We can take a quick look at the DataFrame using the `head()` and `tail()` methods, but also note how Jupyter notebooks will already return a basic view of the DataFrame as long as it is the last thing evaluated in a cell, so mostly we will use that 'shortcut' below.

In [ ]:
df.head(3)

In [ ]:
df.tail(2)

In [ ]:
# the last object evaluated in a cell is outputted by Jupyter

df

If we access an individual column we will get a `Series`. This can also be done with a magic method that treats the column name like a property. The two calls below are mostly equivalent (sometimes the longer string subscript access is required)

In [ ]:
df.script

In [ ]:
df["script"]

Most methods on a DataFrame will return a new DataFrame. Here we pull out the width and height of the pages (leaves) for those manuscripts where the data is recorded. Since that method returns a `DataFrame` we can immediately *chain* another method, `dropna()`, to drop all rows where either value is NA.

NOTE CAREFULLY: The indices for the rows *do not change* (unless we force it to by using a different method). That means our indices are no longer sequential. This is an advantage!

In [ ]:
dimensions = df[["leaf-height", "leaf-width"]].dropna()
dimensions.index

Here we can pull the rows from the original dataframe, but ONLY at the index locations (`.loc`) specified in the `dimensions` dataframe. Note how we now have a dataframe with only 727 entries.

In [ ]:
df.loc[dimensions.index]

### Operating on Columns

We can insert new `Series` objects (or actually even normal lists, or numpy arrays, etc) into the dataframe as columns. Here, we create one column using a vectorized boolean comparison, and another one which we create by multiplying two existing columns. Once again note how these are *vector* operations -- `ColA * ColB` multiplies the matching values in every row, all in one statement.

Note how in the automatic view the reported shape is increasing by one column each time....

In [ ]:
# insert takes index-to-insert, column-name, Series as positional arguments

df.insert(3, "illustrated", df.illustrations > 0)
df

In [ ]:
df.insert(4, "surface", df["leaf-height"] * df["leaf-width"])
df

### Just Checking...

Let's just see if the first entry by *position* (not index, `iloc` always uses sequential position) has been correctly calculated. This is not exhaustive, but it is a good habit to make sure that things look like they are doing what they should be doing.

In [ ]:
dimensions.iloc[0]

In [ ]:
308 * 215 == df.iloc[0]["surface"]

### More index tricks

Now we'll save the new dataframe we get by accessing the original `df` at the indices from the `dimensions` df, into a new variable called `df_valid`

In [43]:
df_valid = df.loc[dimensions.index]

Sorting is easy in pandas, using `sort_values` (this sorts the whole row, using the values in the column you provide)

In [ ]:
df_valid.sort_values(by="surface")

... but notice that some of `surface` entries are 0, because one of `leaf-height` or `leaf-width` was mistakenly entered as 0 instead of NA. Let's clean that up.

The main tool for subsetting by conditions inherently uses a boolean Series under the hood, but you can think of it like this: "return the rows of `df_valid` at the indices where `df_valid.surface` (a column) is greater than zero"

In [ ]:
df_valid = df_valid[df_valid.surface > 0]
df_valid

## Grouping and aggregating

A very powerful method is pandas `groupby()` which creates a special grouping object that can be used for a lot of things. For example, let's see how many illustrated manuscripts occur for each script type...

In [ ]:
df_valid.groupby("script")["illustrated"].value_counts()

We can also "normalize" these numbers by count to obtain comparable proportions

In [ ]:
df_valid.groupby("script")["illustrated"].value_counts(normalize=True)

Finally, and this is just scratching the surface, we can "aggregate" columns from the groups using any function. Common functions (mean, median, std...) can be specific just by their name...

In [ ]:
df_valid.groupby("script")["surface"].agg("mean")

We can subset our valid dataframe again by directly using the boolean column `illustrated` that we created earlier. Now we're down to just 186 manuscripts where we have illustrations as well as valid page sizes.

In [ ]:
df_illustrated = df_valid[df_valid.illustrated]
df_illustrated

### Other formats

Finally, we use pandas again to read the same data from Microsoft Excel.

In [50]:
try:
    import openpyxl
except ImportError:
    !pip install openpyxl

In [ ]:
dfxl = pd.read_excel("../../datasets/arthur/manuscripts.xlsx")
dfxl

Oh no! The Excel indices start from 0, but the CSV we used before started from 1 (maybe it was created in R or Matlab). We can just use our vector operation trick again to add one to the entire index column (ie to each row in it)

In [52]:
dfxl.index += 1

Now (with the indices matching), we use the index locations from our `df_illustrated` dataframe but we grab the rows straight from the new Excel dataframe!

In [ ]:
dfxl.loc[df_illustrated.index]

And finally, a little taste of validation and cleaning. We can use `.equals()` to compare entire `Series` or `DataFrame` objects. Here we find out that the `signature` columns are *not* equal, even though they should be

In [ ]:
dfxl.loc[df_illustrated.index]["signature"].equals(df_illustrated["signature"])

A different method, `.eq()`, returns a boolean `Series` instead (with an equality value for every row). By using a sneaky trick, we assign that to a variable, and then invert it using logical NOT (the tilde `~` operator in pandas). In other words, the original series was `True` almost everywhere and `False` in a couple of places, we swap that, so it is `True` just for the problem rows, and see what indices they are.

Take your time with this idea, it is not simple to understand at first. It can help to make new cells, and then look at individual variables. What is `s`? What does `~s` look like? etc.

In [ ]:
s = dfxl.loc[df_illustrated.index]["signature"].eq(df_illustrated["signature"])
df_illustrated[~s]

... and let's take a look. Oh, this old trick -- someone has accidentally left an extra space at the end of two column entries. This kind of problem comes up all the time in data analysis, and requires constant vigilance. Fixing these problems is called "data cleaning" and,unfortunately, is a necessary process for almost all real-world work.

In [ ]:
dfxl.loc[308]["signature"]

In [ ]:
df_illustrated.loc[308]["signature"]

And to finish, a little trick with Python "f-strings". It is not as complicated as it looks, see if you can figure it out!

(Again, try breaking it into smaller pieces, and then putting them back together!)

In [ ]:
print(
    f"A: {repr(dfxl.loc[596]['signature'])}\nB: {repr(df_illustrated.loc[596]['signature'])}"
)

```
Version History

Current: v1.0.3

10/9/24: 1.0.0: first draft, BN
18/9/24: 1.0.1: add group_by examples, BN
04/10/24: 1.0.2: correct typos and proofread, MK
13/10/24: 1.0.3: move to public, BN
```